<a href="https://colab.research.google.com/github/Tamrika/BigData/blob/main/Detecting_Fake_news_with_Apache_Spark_and_Spark_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Detecting Fake news with Apache Spark and Spark NLP

## Installing Java and Spark NLP

In [ ]:
import os
# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:"+ os.environ["PATH"]
! java -version
# Install pyspark
! pip install --ignore-installed pyspark==2.4.4
# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.6.3

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)
     |████████████████████████████████| 215.7MB 65kB/s 
     |████████████████████████████████| 204kB 30.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130389 sha256=3e06de8a3f71cebbb3d4ecdfead983c40550414038fe53d60d76461273e25d4b
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
     |████████████████████████████████| 133kB 4.9MB/s 


## Importing necessary packages

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
spark=sparknlp.start()
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.6.3
Apache Spark version:  2.4.4


## Create a dataset

In [ ]:
text_list=[
"""Donald Trump a KGB Spy? 11/02/2016 In today’s video, Christopher Greene of AMTV reports Hillary Clinton campaign accusation that Donald Trump is a KGB spy is about as weak and baseless a claim as a Salem witch hunt or McCarthy era trial. It’s only because Hillary Clinton is losing that she is lobbing conspiracy theory. Citizen Quasar The way I see it, one of two things will happen: 1. Trump will win by a landslide but the election will be stolen via electronic voting, just like I have been predicting for over a decade, and the American People will accept the skewed election results just like they accept the TSA into their crotches. 2. Somebody will bust a cap in Hillary’s @$$ killing her and the election will be postponed. Follow AMTV!""",
"""President Barack Obama said former Secretary of State Hillary Clinton’s use of a personal email server was a mistake, but that U.S. national security hadn’t been endangered. In his first extensive remarks on the controversy that has roiled the Democratic presidential primary, Mr. Obama said on CBS’s “60 Minutes” program that questions about Mrs. Clinton’s email arrangement were legitimate. “It is important for her to answer these questions to the satisfaction of the American public,” Mr. Obama said.""",
"""Abby Martin Exposes What Hillary Clinton Really Represents ‹ › Since 2011, VNN has operated as part of the Veterans Today Network ; a group that operates over 50 plus media, information and service online sites for U.S. Military Veterans. Morning Joe Destroys Corrupt Clinton Foundation (Laughable) “Total Corruption” By VNN on October 28, 2016 'Pay for Play' and 'Quid Pro Quo' 'Shut Down The Foundation' Inside the Clinton’s Foundation and Personal Gains They are bragging that they can shake down foundation clients, for Bill Clinton money… This is sleazy… Joe Scarborough. Follow the money.""",
"""President Barack Obama is ramping up efforts to convince individual House members to grant him fast-track authority to negotiate trade deals, focusing his efforts on a dwindling group of undecided Democratic lawmakers.But Democrats who have already backed the deal publicly said these members need to be convinced they are not trading away their own political futures for a vote on fast-track. Potentially decisive are moderate, pro-growth members of the New Democrat Coalition. Its vice-chair, Rep. Jim Himes (D., Conn.), spoke as recently as Monday to the president, after fielding calls from the White House during last week’s recess as well.""",
"""Most American spend over 9 hours a day using media. Is this making us dumber?? Are you living in a media induced trace? Do you know the truth of the world or do you know what the manipulators want you to know?! Professor Jerry Kroth (Ph. D. Psychology) examines the ties between advertising and factual knowledge. Most people can name every mascot of most companies, but they can not name hardly any historical figures. How did we get here? These are questions that Professor Kroth explains in this well thought out presentation. This talk is based on Dr. Kroth's recent book, "Duped! Delusion, Denial, and the end of the American dream." """,
"""Michael Brown’s parents plan to bring a civil lawsuit for the wrongful death of their son against Darren Wilson and the city of Ferguson, Missouri. The announcment came a day after the Justice Department released its report on the abuses of the city’s police department and said Wilson wouldn’t be charged for violating Brown’s civil rights. Brown family lawyers note that the burden of proof is lower in a civil case than the criminal cases that were considered by both the federal government and a St. Louis County grand jury.""",
"""It’s Going to Change RADICALLY With Silver – HUGE Demand Coming | Cliff High Data mining expert Cliff High says the economy is much worse than most people think, and that bubble is going to pop after Election Day. Inflation is also coming, and that will be very positive for precious metals . High contends, “ Gold and silver are going to rise relative to the falling currencies. Gold and silver in actual purchasing power will also rise. They won’t be saying an ounce of gold bought a good suit 100 years ago and an ounce of gold will buy a good suit now. That’s going to change, and it’s also going to change radically with silver . Also, in our data sets between 2019 and 2024 , silver becomes the metal to have… You need to have silver . 2017 Gold Pandas and 2017 Silver Pandas Are Now Available! Secure Your 2017 Panda Coins Today at SD Bullion!""",
"""Senate Majority Leader Mitch McConnell announced a "Plan B" to halt a nuclear deal that would lift sanctions against Iran.The measure, which Republican aides said likely would be voted on Thursday, would prevent President Obama from lifting the sanctions until Iran releases four jailed Americans and recognizes the right of Israel to exist.McConnell made the announcement as Democrats prepared for a second time to filibuster a resolution of disapproval of the nuclear deal. Soon after McConnell's remarks, 42 Democrats again filibustered the resolution, preventing it from getting the 60 votes needed to advance."My strong preference is for Democrats to simply allow an up-or-down vote on the president's Iran deal." McConnell said. "But if they're determined to make that impossible, then at the very least we should be able to provide some protection to Israel and long-overdue relief to Americans who've languished in Iranian custody for years. Either way, this debate will continue." """,
"""TRUMP TSUNAMI INCOMING: What Trump Did In Florida Today Will Make Him President! TRUMP TSUNAMI INCOMING: What Trump Did In Florida Today Will Make Him President! fisher 5 mins ago News Comments Off on TRUMP TSUNAMI INCOMING: What Trump Did In Florida Today Will Make Him President! TRUMP TSUNAMI INCOMING: What Trump Did In Florida Today Will Make Him President! Breaking! Breaking! Bad news for Hillary in Florida. Early voting numbers from Florida are showing that Republicans have cast 17,000 more votes than Democrats. *** 6 days before the Election in 2012, Democrats in Florida cast 39,000 more votes than Republicans. *** Today, six days before the election, Republicans have now cast 17,000 more votes than Democrats. Watch Trump in Miami, FL today: """,
"""Sen. Marco Rubio (R-Fla.) is adding a veteran New Hampshire political operative to his team as he continues mulling a possible 2016 presidential bid, the latest sign that he is seriously preparing to launch a campaign later this year.Jim Merrill, who worked for former GOP presidential nominee Mitt Romney and ran his 2008 and 2012 New Hampshire primary campaigns, joined Rubio’s fledgling campaign on Monday, aides to the senator said.Merrill will be joining Rubio’s Reclaim America PAC to focus on Rubio’s New Hampshire and broader Northeast political operations."Marco has always been well received in New Hampshire, and should he run for president, he would be very competitive there," Terry Sullivan, who runs Reclaim America, said in a statement. "Jim certainly knows how to win in New Hampshire and in the Northeast, and will be a great addition to our team at Reclaim America.”News of Merrill’s hire was first reported by The New York Times.""",
]


In [ ]:
import pandas as pd
text_df = spark.createDataFrame(pd.DataFrame({"text":text_list}))

In [ ]:
#Show schema
text_df.printSchema()

#Verify the count
text_df.count()

root
 |-- text: string (nullable = true)



10

## Pre-processing Pipeline using Spark NLP

In [ ]:
# Preprocessing (Pipeline)
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
use = UniversalSentenceEncoder.pretrained(lang="en").setInputCols(["document"]).setOutputCol("sentence_embeddings")
model_name = 'classifierdl_use_fakenews' # pre-trained model name 
document_classifier = ClassifierDLModel.pretrained(model_name).setInputCols(['document', 'sentence_embeddings']).setOutputCol("prediction")
nlp_Pipeline = Pipeline( stages=[documentAssembler,use,document_classifier])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[OK!]


## Apply the ML Pipeline

In [ ]:
# Training (Pipeline)
empty_df = spark.createDataFrame([['']]).toDF("text") 
pipelineModel = nlp_Pipeline.fit(empty_df)
result_df = pipelineModel.transform(text_df)
# Show Schema
result_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContains

## Visualize the results

In [ ]:
# Visualize the results
import pyspark.sql.functions as F 
result_df.select(F.explode(F.arrays_zip('prediction.result', 'document.result')).alias("cols")) \
          .select(F.expr("cols['0']").alias("prediction"), F.expr("cols['1']").alias("document")).show(truncate=False)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Assignment 5
Rewrite the codes for detecting fake/real news in Trump and Biden tweet dataset.

• Read the article [21]

• Rewrite the codes for download the two files:

  o Use the two links in the article

  o Use the links from the raw data by clicking the raw button in the GitHub pages. If not, you will get html pages instead of csv files. The link should be started with “https://raw.githubusercontent.com/....”

• Rewrite the codes appropriately
• Visualize the results

o Note: performance may not be great, because pre-trained model is based on 2016
election-related datasets in Assignment 6.

## Importing Trump and biden dataset

In [ ]:
import pickle
import cloudpickle as cp
lst_donald_trump_tweets=[]
lst_joe_biden_tweets=[]
from urllib.request import urlopen
lst_donald_trump_tweets = cp.load(urlopen("https://raw.githubusercontent.com/gomachinelearning/Blogs/master/DonaldTrumpTweets.pickle"))
lst_joe_biden_tweets = cp.load(urlopen("https://raw.githubusercontent.com/gomachinelearning/Blogs/master/JoeBidenTweets.pickle"))
len(lst_donald_trump_tweets), len(lst_joe_biden_tweets)

(3237, 3230)

In [ ]:
import pandas as pd
trump_df = spark.createDataFrame(pd.DataFrame({'text':lst_donald_trump_tweets}))
biden_df = spark.createDataFrame(pd.DataFrame({'text':lst_joe_biden_tweets}))

## Print Schema

In [ ]:
trump_df.printSchema()
biden_df.printSchema()

trump_df.count(),biden_df.count()

root
 |-- text: string (nullable = true)

root
 |-- text: string (nullable = true)



(3237, 3230)

## Pre-processing Pipeline using Spark NLP

In [ ]:
# Preprocessing (Pipeline)
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
use = UniversalSentenceEncoder.pretrained(lang="en").setInputCols(["document"]).setOutputCol("sentence_embeddings")
model_name = 'classifierdl_use_fakenews' # pre-trained model name 
document_classifier = ClassifierDLModel.pretrained(model_name).setInputCols(['document', 'sentence_embeddings']).setOutputCol("prediction")
nlp_Pipeline = Pipeline( stages=[documentAssembler,use,document_classifier])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[OK!]


## Applying ML Pipeline

In [ ]:
# Training (Pipeline)
empty_df = spark.createDataFrame([['']]).toDF("text") 
pipelineModel = nlp_Pipeline.fit(empty_df)
result_trump_df = pipelineModel.transform(trump_df)
result_biden_df = pipelineModel.transform(biden_df)
# Show Schema
result_trump_df.printSchema()
result_biden_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContains

## Visualize the results

In [ ]:
# Visualize the results
import pyspark.sql.functions as F 
#Trump dataset results
print("Trump Dataset results:")
result_trump_df.select(F.explode(F.arrays_zip('prediction.result', 'document.result')).alias("cols")) \
          .select(F.expr("cols['0']").alias("prediction"), F.expr("cols['1']").alias("document")).show(truncate=False)
#Biden dataset results
print("Biden Dataset results:")
result_biden_df.select(F.explode(F.arrays_zip('prediction.result', 'document.result')).alias("cols")) \
          .select(F.expr("cols['0']").alias("prediction"), F.expr("cols['1']").alias("document")).show(truncate=False)

Trump Dataset results:
+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|prediction|document                                                                                                                                                                                                                                                                                            |
+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|FAKE      |https://t.co/0LKTMOgk1T                        

## Assignment 6

### Get raw fake news dataset from https://github.com/joolsa/fake_real_news_dataset 

In [ ]:
!wget https://github.com/joolsa/fake_real_news_dataset/blob/master/fake_or_real_news.csv.zip

--2021-04-28 19:59:34--  https://github.com/joolsa/fake_real_news_dataset/blob/master/fake_or_real_news.csv.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘fake_or_real_news.csv.zip’

fake_or_real_news.c     [ <=>                ] 106.45K  --.-KB/s    in 0.04s   

2021-04-28 19:59:35 (2.62 MB/s) - ‘fake_or_real_news.csv.zip’ saved [109006]



### Unzipping the dataset

In [ ]:
import pandas as pd
import zipfile

zf = zipfile.ZipFile('/content/fake_or_real_news.csv.zip') 
df = pd.read_csv(zf.open('fake_or_real_news.csv'))

In [ ]:
df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [ ]:
pd.set_option("display.max_colwidth", -1)
df.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


Unnamed: 0  \
0  8476         
1  10294        
2  3608         
3  10142        
4  875          
5  6903         
6  7341         
7  95           
8  4869         
9  2909         

                                                                                   title  \
0  You Can Smell Hillary’s Fear                                                            
1  Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)   
2  Kerry to go to Paris in gesture of sympathy                                             
3  Bernie supporters on Twitter erupt in anger against the DNC: 'We tried to warn you!'    
4  The Battle of New York: Why This Primary Matters                                        
5  Tehran, USA                                                                             
6  Girl Horrified At What She Watches Boyfriend Do After He Left FaceTime On               
7  ‘Britain’s Schindler’ Dies at 106                                                       
8  Fact check: Trump and Clinton at the 'commander-in-chief' forum                         
9  Iran reportedly makes new push for uranium concessions in nuclear talks                 

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

## Remove unnecessary special characters

In [ ]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–","‘","\n","\r","’","—"]
for char in spec_chars:
    df['text'] = df['text'].str.replace(char, ' ')

In [ ]:
df['text']

0       Daniel Greenfield a Shillman Journalism Fellow at the Freedom Center is a New York writer focusing on radical Islam  In the final stretch of the election Hillary Rodham Clinton has gone to war with the FBI  The word “unprecedented” has been thrown around so often this election that it ought to be retired But it s still unprecedented for the nominee of a major political party to go war with the FBI  But that s exactly what Hillary and her people have done Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary s opponent in this election  The FBI is under attack by everyone from Obama to CNN Hillary s people have circulated a letter attacking Comey There are currently more media hit pieces lambasting him than targeting Trump It wouldn t be too surprising if the Clintons or their allies were to start running attack ads against the FBI  The FBI s leadership is being warned that the entire leftwing 

## Create a spark dataframe

In [ ]:
classification_df = spark.createDataFrame(df)

In [ ]:
classification_df.select('label')

DataFrame[label: string]

In [ ]:
classification_df.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)



In [ ]:
classification_df.count()

6335

## Pre-processing Pipeline using Spark NLP

In [ ]:
# Preprocessing (Pipeline)
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
use = UniversalSentenceEncoder.pretrained(lang="en").setInputCols(["document"]).setOutputCol("sentence_embeddings")
model_name = 'classifierdl_use_fakenews' # pre-trained model name 
document_classifier = ClassifierDLModel.pretrained(model_name).setInputCols(['document', 'sentence_embeddings']).setOutputCol("prediction")
nlp_Pipeline = Pipeline( stages=[documentAssembler,use,document_classifier])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[OK!]


## Apply the ML Pipeline

In [ ]:
# Training (Pipeline)
empty_df = spark.createDataFrame([['']]).toDF("text") 
pipelineModel = nlp_Pipeline.fit(empty_df)
classification_result_df = pipelineModel.transform(classification_df)
# Show Schema
classification_result_df.printSchema()


root
 |-- Unnamed: 0: long (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-

## Visualize the results

In [ ]:
# Visualize the results
import pyspark.sql.functions as F 
classification_result=classification_result_df.select('label',F.explode(F.arrays_zip('prediction.result', 'document.result')).alias("cols")) \
          .select('label',F.expr("cols['0']").alias("prediction"), F.expr("cols['1']").alias("document"))
classification_result.show(truncate=False)

+-----+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Observation:
The following result shows that there are 509 rows that are predicted wrongly

In [ ]:
classification_result.filter(classification_result.label!= classification_result.prediction).count()

509

## Wrongly Predicted data sample

In [ ]:
classification_result.filter(classification_result.label!= classification_result.prediction).show(truncate=False)

+-----+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------